In [1]:
import numpy as np
from simple.ohlc import renko
from simple.chart import chartFigure, chartTrades
from simple.funcs import symlog
from simple.backtest import npBacktestMarket

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)[:100_000]
len(T)

100000

In [3]:
Buy = T[T.Size > 0]
Sell = T[T.Size < 0]
Buy[-5:]

rec.array([('2022-05-08T01:01:46.276000', 2622.12,   2.),
           ('2022-05-08T01:01:46.289000', 2622.12,   2.),
           ('2022-05-08T01:01:46.390000', 2622.12, 406.),
           ('2022-05-08T01:01:46.509000', 2622.12, 192.),
           ('2022-05-08T01:01:46.509000', 2622.12,   8.)],
          dtype=[('DateTime', '<M8[us]'), ('Price', '<f8'), ('Size', '<f8')])

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)[:100_000]
S[-5:]

rec.array([('2022-05-08T01:00:01.874000', 2619.  , 2618.98),
           ('2022-05-08T01:00:01.907000', 2619.2 , 2619.03),
           ('2022-05-08T01:00:01.942000', 2619.2 , 2619.15),
           ('2022-05-08T01:00:01.976000', 2619.35, 2619.15),
           ('2022-05-08T01:00:02.009000', 2619.35, 2619.03)],
          dtype={'names': ['DateTime', 'Ask', 'Bid'], 'formats': ['<M8[us]', '<f4', '<f4'], 'offsets': [0, 8, 16], 'itemsize': 24})

In [5]:
len(T), len(S)

(100000, 100000)

In [6]:
C = renko(T, 15)
C[:5], len(C)

([(0, 17610, 2625.0, 2640.0),
  (17611, 18413, 2655.0, 2640.0),
  (18414, 18420, 2625.0, 2640.0),
  (18421, 18421, 2655.0, 2640.0),
  (18422, 52272, 2640.0, 2625.0)],
 64)

In [7]:
s = 0
Signal = np.zeros(len(S.DateTime))
delay = np.timedelta64(1000, 'ms')

for c in C:
    color = 'green' if c[3] > c[2] else 'red'
    duration = T.DateTime[c[1]] - T.DateTime[c[0]]
    tm = T.DateTime[c[1]] + delay
    k = np.searchsorted(S.DateTime, tm)
    if duration < np.timedelta64(10000, 'μs'):
        if c[3] > c[2]:
            s = 1
        elif c[3] < c[2]:
            s = -1
    if k < len(Signal):
        Signal[k] = s

    print(f'{str(c): <26}', T.DateTime[c[1]], tm, f'{duration: <16}', k, s, color, sep='\t')

(0, 17610, 2625.0, 2640.0)	2022-05-08T00:10:12.393999	2022-05-08T00:10:13.393999	552058999 microseconds	15771	0	green
(17611, 18413, 2655.0, 2640.0)	2022-05-08T00:10:36.481000	2022-05-08T00:10:37.481000	24072001 microseconds	16465	0	red
(18414, 18420, 2625.0, 2640.0)	2022-05-08T00:10:36.481000	2022-05-08T00:10:37.481000	0 microseconds  	16465	1	green
(18421, 18421, 2655.0, 2640.0)	2022-05-08T00:10:36.496000	2022-05-08T00:10:37.496000	0 microseconds  	16465	-1	red
(18422, 52272, 2640.0, 2625.0)	2022-05-08T00:30:48.915000	2022-05-08T00:30:49.915000	1212399000 microseconds	50499	-1	red
(52273, 52353, 2610.0, 2625.0)	2022-05-08T00:30:48.921000	2022-05-08T00:30:49.921000	0 microseconds  	50499	1	green
(52354, 52376, 2640.0, 2625.0)	2022-05-08T00:30:49.108999	2022-05-08T00:30:50.108999	175999 microseconds	50505	1	red
(52377, 52381, 2610.0, 2625.0)	2022-05-08T00:30:49.116999	2022-05-08T00:30:50.116999	8000 microseconds	50505	1	green
(52382, 52382, 2640.0, 2625.0)	2022-05-08T00:30:49.135999	20

In [8]:
trades = npBacktestMarket(S.DateTime.astype(np.int64), S.A, S.B, Signal, 0)
trades

AttributeError: recarray has no attribute A

In [ ]:
# timed chart
fig = chartFigure(rows=2,
    Sell=dict(x=Sell.DateTime, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Size), opacity=0.2),
    Buy=dict(x=Buy.DateTime, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Size), opacity=0.2),
    Ask=dict(x=S.DateTime, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DateTime, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),
    Signal=dict(x=S.DateTime, y=Signal, row=2, color='magenta', shape='hv'),
    **chartTrades(trades)
)
fig

In [ ]:
fig.layout.shapes = [{
    'fillcolor': 'green' if c[3] > c[2] else 'red',
    'line': {'color': 'black', 'width': 2},
    'opacity': 0.15,
    'type': 'rect',
    'x0': str(T.DateTime[int(c[0])]),
    'x1': str(T.DateTime[int(c[1])]),
    'y0': c[2],
    'y1': c[3]
} for c in C]